# Configuration

In [1]:
%load_ext literary.notebook

In [2]:
from functools import lru_cache
from pathlib import Path

from jupyter_core.paths import jupyter_config_path
from traitlets.config import (
    Config,
    ConfigFileNotFound,
    JSONFileConfigLoader,
    PyFileConfigLoader,
)

In [3]:
CONFIG_FILE_NAME = "literary_config"
PROJECT_ROOT_MARKERS = (
    "pyproject.toml",
    "setup.py",
    "setup.cfg",
    ".literary-project",
    f"{CONFIG_FILE_NAME}.json",
    f"{CONFIG_FILE_NAME}.py",
)

In [4]:
def load_config(project_path: Path) -> Config:
    """Load the literary config file, or return an empty config object if missing"""
    paths = [project_path, *jupyter_config_path()]
    loaders = [
        JSONFileConfigLoader(f"{CONFIG_FILE_NAME}.json", path=paths),
        PyFileConfigLoader(f"{CONFIG_FILE_NAME}.py", path=paths),
    ]
    config = Config()
    for loader in loaders:
        try:
            config.update(loader.load_config())
        except ConfigFileNotFound:
            pass

    return config

In [5]:
@lru_cache()
def find_project_path(*search_paths) -> Path:
    """Find the root path of the literary project.

    Searches for known files / directories including
    `.git/`, `.hg/`, or `pyproject.toml`.

    :param search_paths: starting search paths
    :return:
    """
    if not search_paths:
        search_paths = [Path.cwd()]

    visited = set()

    for search_path in search_paths:
        for path in (search_path, *search_path.parents):
            # Avoid re-visiting paths
            if path in visited:
                break
            visited.add(path)

            # Look for any Python package marker
            for marker in PROJECT_ROOT_MARKERS:
                if (path / marker).exists():
                    return path

    raise FileNotFoundError("Couldn't find project path")